In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from yahoo_fin import stock_info as si
from collections import deque

import os
import numpy as np
import pandas as pd
import random

In [3]:
# set seed, so we can get the same results after rerunning several times
np.random.seed(314)
tf.random.set_seed(314)
random.seed(314)

In [4]:
def shuffle_in_unison(a, b):
    # shuffle two arrays in the same way
    state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(state)
    np.random.shuffle(b)

def load_data(ticker, n_steps=50, scale=True, shuffle=True, lookup_step=1, split_by_date=True,
               #adjclose adjusted 調整後終値
                test_size=0.2, feature_columns=['adjclose', 'volume', 'open', 'high', 'low']):
    """
    Loads data from Yahoo Finance source, as well as scaling, shuffling, normalizing and splitting.
    Params:
        ticker (str/pd.DataFrame): the ticker you want to load, examples include AAPL, TSLA, etc.
        n_steps (int): the historical sequence length (i.e window size) used to predict, default is 50
        scale (bool): whether to scale prices from 0 to 1, default is True
        shuffle (bool): whether to shuffle the dataset (both training & testing), default is True
        lookup_step (int): the future lookup step to predict, default is 1 (e.g next day)
        split_by_date (bool): whether we split the dataset into training/testing by date, setting it 
            to False will split datasets in a random way
        test_size (float): ratio for test data, default is 0.2 (20% testing data)
        feature_columns (list): the list of features to use to feed into the model, default is everything grabbed from yahoo_fin
    """
    # see if ticker is already a loaded stock from yahoo finance
    if isinstance(ticker, str):
        # load it from yahoo_fin library
        df = si.get_data(ticker)
    elif isinstance(ticker, pd.DataFrame):
        # already loaded, use it directly
        df = ticker
    else:
        raise TypeError("ticker can be either a str or a `pd.DataFrame` instances")
    # this will contain all the elements we want to return from this function
    result = {}
    # we will also return the original dataframe itself
    result['df'] = df.copy()
    
    # make sure that the passed feature_columns exist in the dataframe
    for col in feature_columns:
        assert col in df.columns, f"'{col}' does not exist in the dataframe."
        
    # add date as a column
    if "date" not in df.columns:
        df["date"] = df.index
        
    if scale:
        column_scaler = {}
        # scale the data (prices) from 0 to 1
        for column in feature_columns:
            scaler = preprocessing.MinMaxScaler()
            df[column] = scaler.fit_transform(np.expand_dims(df[column].values, axis=1))
            column_scaler[column] = scaler
        # add the MinMaxScaler instances to the result returned
        result["column_scaler"] = column_scaler
        
    # add the target column (label) by shifting by `lookup_step`
    df['future'] = df['adjclose'].shift(-lookup_step)
    
    # last `lookup_step` columns contains NaN in future column
    # get them before droping NaNs
    last_sequence = np.array(df[feature_columns].tail(lookup_step))
    
    # drop NaNs
    df.dropna(inplace=True)
    
    sequence_data = []
    sequences = deque(maxlen=n_steps)
    
    for entry, target in zip(df[feature_columns + ["date"]].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == n_steps:
            sequence_data.append([np.array(sequences), target])
            
    # get the last sequence by appending the last `n_step` sequence with `lookup_step` sequence
    # for instance, if n_steps=50 and lookup_step=10, last_sequence should be of 60 (that is 50+10) length
    # this last_sequence will be used to predict future stock prices that are not available in the dataset
    last_sequence = list([s[:len(feature_columns)] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)    
    # add to result
    result['last_sequence'] = last_sequence
    
    # construct the X's and y's
    X, y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        y.append(target)
    # convert to numpy arrays
    X = np.array(X)
    y = np.array(y)
    
    if split_by_date:
        # split the dataset into training & testing sets by date (not randomly splitting)
        train_samples = int((1 - test_size) * len(X))
        result["X_train"] = X[:train_samples]
        result["y_train"] = y[:train_samples]
        result["X_test"]  = X[train_samples:]
        result["y_test"]  = y[train_samples:]
        
        if shuffle:
            # shuffle the datasets for training (if shuffle parameter is set)
            shuffle_in_unison(result["X_train"], result["y_train"])
            shuffle_in_unison(result["X_test"], result["y_test"])
            
    else:    
        # split the dataset randomly
        result["X_train"], result["X_test"], result["y_train"], result["y_test"] = train_test_split(X, y, 
                                                                                test_size=test_size, shuffle=shuffle)
    # get the list of test set dates
    dates = result["X_test"][:, -1, -1]
    # retrieve test features from the original dataframe
    result["test_df"] = result["df"].loc[dates]
    # remove duplicated dates in the testing dataframe
    result["test_df"] = result["test_df"][~result["test_df"].index.duplicated(keep='first')]
    # remove dates from the training/testing sets & convert to float32
    result["X_train"] = result["X_train"][:, :, :len(feature_columns)].astype(np.float32)
    result["X_test"] = result["X_test"][:, :, :len(feature_columns)].astype(np.float32)
    return result

In [5]:
def create_model(sequence_length, n_features, units=256, cell=LSTM, n_layers=2, dropout=0.3,
                loss="mean_absolute_error", optimizer="rmsprop", bidirectional=False):
    model = Sequential()
    for i in range(n_layers):
        if i == 0:
            # first layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True), batch_input_shape=(None, sequence_length, n_features)))
            else:
                model.add(cell(units, return_sequences=True, batch_input_shape=(None, sequence_length, n_features)))
        elif i == n_layers - 1:
            # last layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=False)))
            else:
                model.add(cell(units, return_sequences=False))
        else:
            # hidden layers
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True)))
            else:
                model.add(cell(units, return_sequences=True))
        # add dropout after each layer
        model.add(Dropout(dropout))
    model.add(Dense(1, activation="linear"))
    model.compile(loss=loss, metrics=["mean_absolute_error"], optimizer=optimizer)
    return model

In [6]:
import os
import time
from tensorflow.keras.layers import LSTM

# Window size or the sequence length
N_STEPS = 50
# Lookup step, 1 is the next day
LOOKUP_STEP = 15
# whether to scale feature columns & output price as well
SCALE = True
scale_str = f"sc-{int(SCALE)}"
# whether to shuffle the dataset
SHUFFLE = True
shuffle_str = f"sh-{int(SHUFFLE)}"
# whether to split the training/testing set by date
SPLIT_BY_DATE = False
split_by_date_str = f"sbd-{int(SPLIT_BY_DATE)}"
# test ratio size, 0.2 is 20%
TEST_SIZE = 0.2
# features to use
FEATURE_COLUMNS = ["adjclose", "volume", "open", "high", "low"]
# date now
date_now = time.strftime("%Y-%m-%d")
### model parameters
N_LAYERS = 2
# LSTM cell
CELL = LSTM
# 256 LSTM neurons
UNITS = 256
# 40% dropout
DROPOUT = 0.4
# whether to use bidirectional RNNs
BIDIRECTIONAL = False
### training parameters
# mean absolute error loss
# LOSS = "mae"
# huber loss
LOSS = "huber_loss"
OPTIMIZER = "adam"
BATCH_SIZE = 64
EPOCHS = 500
# tesla stock market
ticker = "TSLA"
ticker_data_filename = os.path.join("data", f"{ticker}_{date_now}.csv")
# model name to save, making it as unique as possible based on parameters
model_name = f"{date_now}_{ticker}-{shuffle_str}-{scale_str}-{split_by_date_str}-\
{LOSS}-{OPTIMIZER}-{CELL.__name__}-seq-{N_STEPS}-step-{LOOKUP_STEP}-layers-{N_LAYERS}-units-{UNITS}"
if BIDIRECTIONAL:
    model_name += "-b"

In [7]:
# create these folders if they does not exist
if not os.path.isdir("results"):
    os.mkdir("results")
if not os.path.isdir("logs"):
    os.mkdir("logs")
if not os.path.isdir("data"):
    os.mkdir("data")

In [8]:
# load the data
data = load_data(ticker, N_STEPS, scale=SCALE, split_by_date=SPLIT_BY_DATE, 
                shuffle=SHUFFLE, lookup_step=LOOKUP_STEP, test_size=TEST_SIZE, 
                feature_columns=FEATURE_COLUMNS)
# save the dataframe
data["df"].to_csv(ticker_data_filename)
# construct the model
model = create_model(N_STEPS, len(FEATURE_COLUMNS), loss=LOSS, units=UNITS, cell=CELL, n_layers=N_LAYERS,
                    dropout=DROPOUT, optimizer=OPTIMIZER, bidirectional=BIDIRECTIONAL)
# some tensorflow callbacks
checkpointer = ModelCheckpoint(os.path.join("results", model_name + ".h5"), save_weights_only=True, save_best_only=True, verbose=1)
tensorboard = TensorBoard(log_dir=os.path.join("logs", model_name))
# train the model and save the weights whenever we see 
# a new optimal model using ModelCheckpoint
history = model.fit(data["X_train"], data["y_train"],
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(data["X_test"], data["y_test"]),
                    callbacks=[checkpointer, tensorboard],
                    verbose=1)

Epoch 1/500
34/34 [==============================] - 6s 40ms/step - loss: 0.0055 - mean_absolute_error: 0.0455 - val_loss: 7.2878e-04 - val_mean_absolute_error: 0.0272

Epoch 00001: val_loss improved from inf to 0.00073, saving model to results\2021-03-31_TSLA-sh-1-sc-1-sbd-0-huber_loss-adam-LSTM-seq-50-step-15-layers-2-units-256.h5
Epoch 2/500
34/34 [==============================] - 1s 15ms/step - loss: 8.5324e-04 - mean_absolute_error: 0.0211 - val_loss: 3.8083e-04 - val_mean_absolute_error: 0.0145

Epoch 00002: val_loss improved from 0.00073 to 0.00038, saving model to results\2021-03-31_TSLA-sh-1-sc-1-sbd-0-huber_loss-adam-LSTM-seq-50-step-15-layers-2-units-256.h5
Epoch 3/500
34/34 [==============================] - 1s 15ms/step - loss: 7.6647e-04 - mean_absolute_error: 0.0185 - val_loss: 4.9115e-04 - val_mean_absolute_error: 0.0127

Epoch 00003: val_loss did not improve from 0.00038
Epoch 4/500
34/34 [==============================] - 1s 15ms/step - loss: 0.0011 - mean_absolute_e

Epoch 35/500
34/34 [==============================] - 1s 15ms/step - loss: 6.7770e-04 - mean_absolute_error: 0.0181 - val_loss: 7.1121e-04 - val_mean_absolute_error: 0.0139

Epoch 00035: val_loss did not improve from 0.00034
Epoch 36/500
34/34 [==============================] - 1s 15ms/step - loss: 6.4497e-04 - mean_absolute_error: 0.0163 - val_loss: 4.0556e-04 - val_mean_absolute_error: 0.0167

Epoch 00036: val_loss did not improve from 0.00034
Epoch 37/500
34/34 [==============================] - 1s 15ms/step - loss: 6.0883e-04 - mean_absolute_error: 0.0185 - val_loss: 3.6969e-04 - val_mean_absolute_error: 0.0122

Epoch 00037: val_loss did not improve from 0.00034
Epoch 38/500
34/34 [==============================] - 1s 15ms/step - loss: 7.2612e-04 - mean_absolute_error: 0.0170 - val_loss: 3.9910e-04 - val_mean_absolute_error: 0.0127

Epoch 00038: val_loss did not improve from 0.00034
Epoch 39/500
34/34 [==============================] - 1s 15ms/step - loss: 5.8523e-04 - mean_absolut

34/34 [==============================] - 1s 15ms/step - loss: 5.9445e-04 - mean_absolute_error: 0.0151 - val_loss: 3.7349e-04 - val_mean_absolute_error: 0.0119

Epoch 00067: val_loss did not improve from 0.00028
Epoch 68/500
34/34 [==============================] - 1s 15ms/step - loss: 5.3764e-04 - mean_absolute_error: 0.0150 - val_loss: 3.6184e-04 - val_mean_absolute_error: 0.0105

Epoch 00068: val_loss did not improve from 0.00028
Epoch 69/500
34/34 [==============================] - 1s 15ms/step - loss: 5.8101e-04 - mean_absolute_error: 0.0156 - val_loss: 5.3079e-04 - val_mean_absolute_error: 0.0136

Epoch 00069: val_loss did not improve from 0.00028
Epoch 70/500
34/34 [==============================] - 1s 15ms/step - loss: 6.1019e-04 - mean_absolute_error: 0.0179 - val_loss: 3.0126e-04 - val_mean_absolute_error: 0.0106

Epoch 00070: val_loss did not improve from 0.00028
Epoch 71/500
34/34 [==============================] - 1s 15ms/step - loss: 5.2237e-04 - mean_absolute_error: 0.01

34/34 [==============================] - 1s 16ms/step - loss: 4.3970e-04 - mean_absolute_error: 0.0151 - val_loss: 2.4384e-04 - val_mean_absolute_error: 0.0110

Epoch 00101: val_loss did not improve from 0.00020
Epoch 102/500
34/34 [==============================] - 1s 16ms/step - loss: 3.6403e-04 - mean_absolute_error: 0.0137 - val_loss: 2.0144e-04 - val_mean_absolute_error: 0.0094

Epoch 00102: val_loss did not improve from 0.00020
Epoch 103/500
34/34 [==============================] - 1s 16ms/step - loss: 3.4073e-04 - mean_absolute_error: 0.0137 - val_loss: 3.4396e-04 - val_mean_absolute_error: 0.0101

Epoch 00103: val_loss did not improve from 0.00020
Epoch 104/500
34/34 [==============================] - 1s 16ms/step - loss: 3.5257e-04 - mean_absolute_error: 0.0129 - val_loss: 5.9536e-04 - val_mean_absolute_error: 0.0153

Epoch 00104: val_loss did not improve from 0.00020
Epoch 105/500
34/34 [==============================] - 1s 16ms/step - loss: 0.0015 - mean_absolute_error: 0.02

34/34 [==============================] - 1s 16ms/step - loss: 4.1170e-04 - mean_absolute_error: 0.0142 - val_loss: 2.0179e-04 - val_mean_absolute_error: 0.0094

Epoch 00136: val_loss did not improve from 0.00016
Epoch 137/500
34/34 [==============================] - 1s 16ms/step - loss: 5.6318e-04 - mean_absolute_error: 0.0168 - val_loss: 1.8617e-04 - val_mean_absolute_error: 0.0102

Epoch 00137: val_loss did not improve from 0.00016
Epoch 138/500
34/34 [==============================] - 1s 15ms/step - loss: 3.7930e-04 - mean_absolute_error: 0.0134 - val_loss: 1.4350e-04 - val_mean_absolute_error: 0.0080

Epoch 00138: val_loss improved from 0.00016 to 0.00014, saving model to results\2021-03-31_TSLA-sh-1-sc-1-sbd-0-huber_loss-adam-LSTM-seq-50-step-15-layers-2-units-256.h5
Epoch 139/500
34/34 [==============================] - 1s 15ms/step - loss: 2.9582e-04 - mean_absolute_error: 0.0124 - val_loss: 1.6100e-04 - val_mean_absolute_error: 0.0083

Epoch 00139: val_loss did not improve from

...

34/34 [==============================] - 1s 16ms/step - loss: 2.2862e-04 - mean_absolute_error: 0.0114 - val_loss: 8.8194e-05 - val_mean_absolute_error: 0.0072

Epoch 00351: val_loss did not improve from 0.00007
Epoch 352/500
34/34 [==============================] - 1s 15ms/step - loss: 1.8841e-04 - mean_absolute_error: 0.0109 - val_loss: 8.5177e-05 - val_mean_absolute_error: 0.0079

Epoch 00352: val_loss did not improve from 0.00007
Epoch 353/500
34/34 [==============================] - 1s 16ms/step - loss: 1.8450e-04 - mean_absolute_error: 0.0111 - val_loss: 1.4524e-04 - val_mean_absolute_error: 0.0073

Epoch 00353: val_loss did not improve from 0.00007
Epoch 354/500
34/34 [==============================] - 1s 16ms/step - loss: 2.2177e-04 - mean_absolute_error: 0.0115 - val_loss: 1.6108e-04 - val_mean_absolute_error: 0.0080

Epoch 00354: val_loss did not improve from 0.00007
Epoch 355/500
34/34 [==============================] - 1s 16ms/step - loss: 1.7260e-04 - mean_absolute_error: 

34/34 [==============================] - 1s 16ms/step - loss: 7.2836e-04 - mean_absolute_error: 0.0174 - val_loss: 2.2717e-04 - val_mean_absolute_error: 0.0111

Epoch 00387: val_loss did not improve from 0.00007
Epoch 388/500
34/34 [==============================] - 1s 16ms/step - loss: 3.4401e-04 - mean_absolute_error: 0.0153 - val_loss: 1.9144e-04 - val_mean_absolute_error: 0.0089

Epoch 00388: val_loss did not improve from 0.00007
Epoch 389/500
34/34 [==============================] - 1s 16ms/step - loss: 3.5456e-04 - mean_absolute_error: 0.0136 - val_loss: 2.1515e-04 - val_mean_absolute_error: 0.0106

Epoch 00389: val_loss did not improve from 0.00007
Epoch 390/500
34/34 [==============================] - 1s 16ms/step - loss: 3.3728e-04 - mean_absolute_error: 0.0138 - val_loss: 1.7863e-04 - val_mean_absolute_error: 0.0090

Epoch 00390: val_loss did not improve from 0.00007
Epoch 391/500
34/34 [==============================] - 1s 16ms/step - loss: 2.3776e-04 - mean_absolute_error: 

34/34 [==============================] - 1s 16ms/step - loss: 1.4544e-04 - mean_absolute_error: 0.0095 - val_loss: 7.2323e-05 - val_mean_absolute_error: 0.0076

Epoch 00423: val_loss did not improve from 0.00007
Epoch 424/500
34/34 [==============================] - 1s 15ms/step - loss: 1.5725e-04 - mean_absolute_error: 0.0099 - val_loss: 1.6730e-04 - val_mean_absolute_error: 0.0098

Epoch 00424: val_loss did not improve from 0.00007
Epoch 425/500
34/34 [==============================] - 1s 16ms/step - loss: 1.8889e-04 - mean_absolute_error: 0.0114 - val_loss: 1.3062e-04 - val_mean_absolute_error: 0.0087

Epoch 00425: val_loss did not improve from 0.00007
Epoch 426/500
34/34 [==============================] - 1s 16ms/step - loss: 4.5151e-04 - mean_absolute_error: 0.0141 - val_loss: 2.1779e-04 - val_mean_absolute_error: 0.0100

Epoch 00426: val_loss did not improve from 0.00007
Epoch 427/500
34/34 [==============================] - 1s 16ms/step - loss: 3.4627e-04 - mean_absolute_error: 

34/34 [==============================] - 1s 16ms/step - loss: 1.2454e-04 - mean_absolute_error: 0.0089 - val_loss: 5.5314e-05 - val_mean_absolute_error: 0.0065

Epoch 00457: val_loss did not improve from 0.00005
Epoch 458/500
34/34 [==============================] - 1s 16ms/step - loss: 1.4920e-04 - mean_absolute_error: 0.0096 - val_loss: 9.1839e-05 - val_mean_absolute_error: 0.0082

Epoch 00458: val_loss did not improve from 0.00005
Epoch 459/500
34/34 [==============================] - 1s 16ms/step - loss: 1.3015e-04 - mean_absolute_error: 0.0091 - val_loss: 6.5835e-05 - val_mean_absolute_error: 0.0063

Epoch 00459: val_loss did not improve from 0.00005
Epoch 460/500
34/34 [==============================] - 1s 16ms/step - loss: 1.4918e-04 - mean_absolute_error: 0.0094 - val_loss: 7.0654e-05 - val_mean_absolute_error: 0.0066

Epoch 00460: val_loss did not improve from 0.00005
Epoch 461/500
34/34 [==============================] - 1s 16ms/step - loss: 1.5177e-04 - mean_absolute_error: 

34/34 [==============================] - 1s 16ms/step - loss: 1.2528e-04 - mean_absolute_error: 0.0092 - val_loss: 5.3768e-05 - val_mean_absolute_error: 0.0059

Epoch 00493: val_loss improved from 0.00005 to 0.00005, saving model to results\2021-03-31_TSLA-sh-1-sc-1-sbd-0-huber_loss-adam-LSTM-seq-50-step-15-layers-2-units-256.h5
Epoch 494/500
34/34 [==============================] - 1s 16ms/step - loss: 1.6340e-04 - mean_absolute_error: 0.0093 - val_loss: 8.6271e-05 - val_mean_absolute_error: 0.0072

Epoch 00494: val_loss did not improve from 0.00005
Epoch 495/500
34/34 [==============================] - 1s 16ms/step - loss: 1.7590e-04 - mean_absolute_error: 0.0100 - val_loss: 8.8044e-05 - val_mean_absolute_error: 0.0082

Epoch 00495: val_loss did not improve from 0.00005
Epoch 496/500
34/34 [==============================] - 1s 16ms/step - loss: 1.2848e-04 - mean_absolute_error: 0.0096 - val_loss: 1.1213e-04 - val_mean_absolute_error: 0.0074

Epoch 00496: val_loss did not improve from